# Data Processing
##### Sean Wade

In [151]:
import pandas as pd
import numpy as np
import json
from haversine import haversine

from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://seanwade.com/jupyter.css').read())

## Load Data

In [4]:
business = pd.read_csv('../raw_data/csv/yelp_academic_dataset_business.csv')
review = pd.read_csv('../raw_data/csv/yelp_academic_dataset_review.csv')
user = pd.read_csv('../raw_data/csv/yelp_academic_dataset_user.csv')

/Users/seanwade/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,4,7,17,26,29,49,60,62,79,86,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Add business name to dataframes
business_dict = dict(zip(business['business_id'], business['name']))
review['biz_name'] = [business_dict[x] for x in review['business_id'].tolist()]

In [ ]:
# for calc distance
business['loc'] = list(zip(business.latitude, business.longitude))

## All Cities

In [160]:
state_set = set()
print "City, Num Bizz"
for x in business.state.unique():
    if len(getState(business, x)) > 100: 
        state_set.add(x)
        print x, len(getState(business, x))

City, Num Bizz
PA 4086
NC 6835
SC 325
WI 3066
IL 808
AZ 36500
NV 23591
QC 5591
ON 530
EDH 3297
MLN 161
BW 1055


## All Types of businesses

In [79]:
category_set = set()
for x in business['categories'].tolist():
    for y in eval(x):
        category_set.add(y.lower())

In [80]:
category_set

{'emergency pet hospital',
 'irrigation',
 'fondue',
 'dance clubs',
 'chinese',
 'hats',
 'engraving',
 'desserts',
 'farms',
 'tai chi',
 'tours',
 'gay bars',
 'unofficial yelp events',
 'feng shui',
 'popcorn shops',
 'nurseries & gardening',
 'drive-thru bars',
 'aquarium services',
 'vietnamese',
 'furniture repair',
 'art museums',
 'tattoo removal',
 'painters',
 'pediatric dentists',
 'country clubs',
 'courthouses',
 'dim sum',
 'herbs & spices',
 'traffic ticketing law',
 'university housing',
 'animal shelters',
 'chiropractors',
 'egyptian',
 'propane',
 'automotive',
 'mobile dent repair',
 'nail technicians',
 'vegan',
 'adult education',
 'real estate agents',
 'bubble tea',
 'hearing aid providers',
 'windshield installation & repair',
 'kids activities',
 'bistros',
 'pumpkin patches',
 'urologists',
 'food stands',
 'check cashing/pay-day loans',
 'haunted houses',
 'orthopedists',
 'bed & breakfast',
 'financial services',
 'real estate',
 'day camps',
 'poutineries

## Get Specific Data

In [134]:
def getCategory(df, category):
    return df[df['categories'].apply(lambda x: category in x.lower())]

In [132]:
def getState(df, state):
    return df[df['state'] == state]

In [135]:
def getClose(df, center, max_dist):
    ''' Get close buisnesses within a radius. '''
    return df[df['loc'].apply(lambda x: haversine(center, x)) <= max_dist]

In [167]:
def plotStarDistribution(df):
    try:
        df.groupby('stars').size().plot(kind='bar')    
    except:
        print "No Matching Results..."

In [131]:
getClose(business, (40.3541155, -80.0146597), .3)

,attributes.Ambience.divey,attributes.Dietary Restrictions.vegan,attributes.Happy Hour,hours.Thursday.open,attributes.Order at Counter,attributes.Hair Types Specialized In.africanamerican,attributes.Hair Types Specialized In.kids,attributes.BYOB,hours.Friday.open,attributes.Good For.latenight,...,attributes.Smoking,attributes.Attire,attributes.Hair Types Specialized In.curly,attributes.Good For Groups,neighborhoods,attributes.Open 24 Hours,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale,loc
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,"(40.3541155, -80.0146597)"
73277,NaN,NaN,NaN,10:00,NaN,NaN,NaN,NaN,10:00,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,"(40.3536720227, -80.0161798167)"
74315,False,NaN,NaN,10:00,NaN,NaN,NaN,NaN,10:00,False,...,NaN,casual,NaN,True,[],NaN,False,NaN,False,"(40.3540905, -80.0146199)"


In [174]:
for state in state_set:
    state_bizz = getState(business, state)
    cat_bizz = getCategory(state_bizz, 'fashion')
    print state
    print cat_bizz.groupby('stars').size()
    print '\n'

ON
stars
2.5    1
3.0    1
3.5    1
4.5    3
dtype: int64


EDH
stars
1.5     2
2.0     3
2.5    20
3.0    34
3.5    53
4.0    71
4.5    40
5.0     6
dtype: int64


NC
stars
1.5     8
2.0    17
2.5    36
3.0    48
3.5    85
4.0    72
4.5    35
5.0    15
dtype: int64


MLN
stars
1.5    1
4.0    1
dtype: int64


WI
stars
1.5     4
2.0    10
2.5    14
3.0    14
3.5    17
4.0    27
4.5    23
5.0     7
dtype: int64


BW
stars
1.5    2
2.0    1
2.5    5
3.0    5
3.5    8
4.0    9
4.5    5
5.0    4
dtype: int64


QC
stars
1.0     1
1.5     4
2.0     6
2.5     9
3.0    23
3.5    37
4.0    37
4.5    24
5.0    17
dtype: int64


PA
stars
1.5     3
2.0     6
2.5     3
3.0    12
3.5    26
4.0    23
4.5    28
5.0    14
dtype: int64


IL
stars
1.0    1
1.5    1
2.0    1
2.5    4
3.0    5
3.5    8
4.0    5
4.5    7
5.0    2
dtype: int64


SC
stars
2.5    2
3.5    2
4.5    1
dtype: int64


AZ
stars
1.0      7
1.5     18
2.0     62
2.5    153
3.0    204
3.5    270
4.0    244
4.5    168
5.0    141
dtype:

In [176]:
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('smwade', "a8zcz2zggpeP7MhyykAS")

In [200]:
def plotCategory(df, category, state):
    df = getCategory(df, category)
    df = getState(df, state)
    
    mapbox_access_token = 'pk.eyJ1Ijoic213YWRlIiwiYSI6ImNpdzU4MmR6ZDAwbGwyeHIzcWRkeHNmeWIifQ.dyJ-FEl459ry0ebLOy8f9g'

    data = Data([
        Scattermapbox(
            name="%s businesses" % category,
            lat=df['latitude'],
            lon=df['longitude'],
            mode='markers',
            marker=Marker(
                size=5,
                color='red',
                opacity=0.5,
            ),
            text=df['name'],
            hoverinfo='text',
        )     
    ])

    layout = Layout(
        title="%s business" % category,
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=df['latitude'].iloc[0],
                lon=df['longitude'].iloc[0]
            ),
            pitch=0,
            zoom=13
        ),
    )

    fig = dict(data=data, layout=layout)
    return py.iplot(fig, filename='temp', validate=False)

In [205]:
plotCategory(business, 'seafood', 'QC')

In [ ]:
cut = np.linspace(0, review['votes.useful'].max(), 10)
group = review.groupby(pd.cut(review['votes.useful'], cut))
print group

In [ ]:
group.agg({'user_id': np.size, 'stars': np.mean}).plot(kind='bar',y='stars')